In [70]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
from sklearn.svm import LinearSVC
%matplotlib inline
import time
from sklearn.neighbors import KNeighborsClassifier

In [71]:
train = pd.read_csv('D:\\Kaggle\\hackerearth\\Dataset1\\Dataset\\train.csv')
train["damage_grade"] = train["damage_grade"].str.replace("Grade ", "")
train[["damage_grade"]] = train[["damage_grade"]].astype(int) 

In [72]:
build_own = pd.read_csv('D:\\Kaggle\\hackerearth\\Dataset1\\Dataset\\Building_Ownership_Use.csv')
print(build_own.head())
build_struc = pd.read_csv('D:\\Kaggle\\hackerearth\\Dataset1\\Dataset\\Building_Structure.csv')
print(build_struc.head())


  building_id  district_id  vdcmun_id  ward_id legal_ownership_status  \
0  a3380c4f75            7        701    70102                Private   
1  a3380c4fd9            7        701    70102                Private   
2  a3380c503e            7        701    70102                Private   
3  a338a4e5f2            7        701    70103                Private   
4  a338a4e653            7        701    70103                Private   

   count_families  has_secondary_use  has_secondary_use_agriculture  \
0             1.0                0.0                              0   
1             1.0                0.0                              0   
2             1.0                0.0                              0   
3             1.0                0.0                              0   
4             1.0                0.0                              0   

   has_secondary_use_hotel  has_secondary_use_rental  \
0                        0                         0   
1                     

In [73]:
merged = pd.merge(build_own, build_struc, on = ['building_id', 'district_id', 'vdcmun_id','ward_id' ])

In [74]:
train_df = pd.merge(train, merged,how = 'left', on = ['building_id', 'district_id', 'vdcmun_id' ])

In [75]:
train_df = train_df.fillna(0)
print (train_df.isnull().sum())

area_assesed                              0
building_id                               0
damage_grade                              0
district_id                               0
has_geotechnical_risk                     0
has_geotechnical_risk_fault_crack         0
has_geotechnical_risk_flood               0
has_geotechnical_risk_land_settlement     0
has_geotechnical_risk_landslide           0
has_geotechnical_risk_liquefaction        0
has_geotechnical_risk_other               0
has_geotechnical_risk_rock_fall           0
has_repair_started                        0
vdcmun_id                                 0
ward_id                                   0
legal_ownership_status                    0
count_families                            0
has_secondary_use                         0
has_secondary_use_agriculture             0
has_secondary_use_hotel                   0
has_secondary_use_rental                  0
has_secondary_use_institution             0
has_secondary_use_school        

In [76]:
train_df.dtypes

area_assesed                               object
building_id                                object
damage_grade                                int32
district_id                                 int64
has_geotechnical_risk                     float64
has_geotechnical_risk_fault_crack           int64
has_geotechnical_risk_flood                 int64
has_geotechnical_risk_land_settlement       int64
has_geotechnical_risk_landslide             int64
has_geotechnical_risk_liquefaction          int64
has_geotechnical_risk_other                 int64
has_geotechnical_risk_rock_fall             int64
has_repair_started                        float64
vdcmun_id                                   int64
ward_id                                     int64
legal_ownership_status                     object
count_families                            float64
has_secondary_use                         float64
has_secondary_use_agriculture               int64
has_secondary_use_hotel                     int64


In [77]:
y = train_df[['damage_grade']]
y.head()

,damage_grade
0,4
1,2
2,1
3,5
4,3


In [78]:
# X.plan_configuration.unique()

In [80]:
X = train_df.drop(['damage_grade','building_id'],axis=1)

In [81]:
cat_type_position =['Not attached', 'Attached-1 side', 'Attached-2 side','Attached-3 side']
cat_type_position_lsc = ['Flat', 'Steep slope', 'Moderate slope']
X['position'] = X['position'].astype("category",ordered=True,categories=cat_type_position).cat.codes
X['land_surface_condition'] = X['land_surface_condition'].astype("category",ordered=True,categories=cat_type_position_lsc).cat.codes
print (X['position'].unique())
print (X['land_surface_condition'].unique())

[0 1 2 3]
[0 1 2]


In [88]:
X['age_range'] = pd.cut(X.age_building,bins=100, include_lowest=False,labels=range(100))

In [89]:
print(X.age_range.unique())
X['age_range'] = X['position'].astype("category",ordered=True,categories=X.age_range.unique()).cat.codes
X['age_range'].unique()


[2, 3, 0, 4, 5, ..., 17, 16, 15, 19, 13]
Length: 22
Categories (22, int64): [0 < 1 < 2 < 3 ... 18 < 19 < 20 < 99]


D:\DataScience\Anaconda\lib\site-packages\pandas\core\internals.py:446: RuntimeWarning: Values and categories have different dtypes. Did you mean to use
'Categorical.from_codes(codes, categories)'?
  return self.make_block(Categorical(self.values, **kwargs))


array([2, 5, 0, 1], dtype=int64)

In [90]:
X = X.drop('age_building',axis =1)

In [91]:
listofcats = ['area_assesed','condition_post_eq','foundation_type','roof_type','ground_floor_type','other_floor_type','plan_configuration','legal_ownership_status'] 
for i in listofcats:
    X[i] = X[i].astype("category").cat.codes

In [92]:
print(X.shape)
X['count_floors'] = X['count_floors_post_eq'] - X['count_floors_pre_eq'] 
X['height_ft'] = X['height_ft_post_eq'] - X['height_ft_pre_eq']
X = X.drop(['count_floors_post_eq','count_floors_pre_eq','height_ft_post_eq','height_ft_pre_eq'], axis = 1)
print(X.shape)

(631761, 51)
(631761, 49)


In [93]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=19)

In [95]:
X_train.age_range.unique()

array([2, 5, 0, 1], dtype=int64)

In [13]:
from sklearn.tree import DecisionTreeClassifier
dtree = DecisionTreeClassifier(max_depth=15)
s = time.time()
print("Training...")
dtree.fit(X_train, y_train.values.ravel())
print("Training Time" + str(time.time() -s ))
s= time.time()
print("Predicting...")
print("DTree Score- "+ str(dtree.score(X_test, y_test)))
print("Predicting Time" + str(time.time() -s ))

Training...
Training Time18.795195817947388
Predicting...
DTree Score- 0.729445098112
Predicting Time0.8556060791015625


In [96]:
from sklearn.ensemble import RandomForestClassifier
rforest = RandomForestClassifier(n_estimators =100,n_jobs=-1,max_features = 0.3)
s = time.time()
print("Training...")
rforest.fit(X_train, y_train.values.ravel())
print("Training Time" + str(time.time() -s ))
s= time.time()
print("Predicting...")
print("DTree Score- "+ str(rforest.score(X_test, y_test)))
print("Predicting Time" + str(time.time() -s ))

Training...
Training Time127.794673204422
Predicting...
DTree Score- 0.761566831461
Predicting Time10.463439464569092


In [97]:
from sklearn.metrics import f1_score
accuracy = f1_score(y_test, rforest.predict(X_test), average='weighted')
print (accuracy)

0.762249141857


In [38]:
print(accuracy)

0.731353711045


In [98]:
#lost test data
test = pd.read_csv('D:\\Kaggle\\hackerearth\\Dataset1\\Dataset\\test.csv')
test.shape

(421175, 13)

In [99]:
test_df = pd.merge(test, merged,how = 'left', on = ['building_id', 'district_id', 'vdcmun_id' ])
test_df.shape

(421175, 52)

In [100]:
test_df['age_range'] = pd.cut(test_df.age_building,bins=100, include_lowest=False,labels=range(100))
print(test_df.age_range.unique())
test_df['age_range'] = test_df['position'].astype("category",ordered=True,categories=test_df.age_range.unique()).cat.codes
test_df['age_range'].unique()
test_df = test_df.drop('age_building',axis =1)

[2, 3, 4, 1, 0, ..., 18, 14, 16, 17, 13]
Length: 22
Categories (22, int64): [0 < 1 < 2 < 3 ... 18 < 19 < 20 < 99]


In [101]:
test_df['position'] = test_df['position'].astype("category",ordered=True,categories=cat_type_position).cat.codes
test_df['land_surface_condition'] = test_df['land_surface_condition'].astype("category",ordered=True,categories=cat_type_position_lsc).cat.codes
print (test_df['position'].unique())
print (test_df['land_surface_condition'].unique())

[1 0 2 3]
[2 1 0]


In [102]:
#listofcats = ['area_assesed','condition_post_eq','land_surface_condition','foundation_type','roof_type','ground_floor_type','other_floor_type','position','plan_configuration','legal_ownership_status'] 
for i in listofcats:
    test_df[i] = test_df[i].astype("category").cat.codes

In [103]:
print(test_df.shape)
test_df['count_floors'] = test_df['count_floors_post_eq'] - test_df['count_floors_pre_eq'] 
test_df['height_ft'] = test_df['height_ft_post_eq'] - test_df['height_ft_pre_eq']
test_df = test_df.drop(['count_floors_post_eq','count_floors_pre_eq','height_ft_post_eq','height_ft_pre_eq'], axis = 1)
print(test_df.shape)

(421175, 52)
(421175, 50)


In [104]:
test_df = test_df.drop('building_id',axis =1) 

In [53]:
#fitting the entire train data
dtree_alltrain = DecisionTreeClassifier(max_depth=20)
s = time.time()
print("Training...")
dtree_alltrain.fit(X, y.values.ravel())
print("Training Time" + str(time.time() -s ))

Training...
Training Time26.693943977355957


In [60]:
#creating the solution dataframe
test_df = test_df.fillna(0)
pred = dtree_alltrain.predict(test_df)
sol_1 = pd.DataFrame({"building_id":test.building_id, "damage_grade":pred})
sol_1['damage_grade'] = 'Grade ' + sol_1['damage_grade'].astype(str)

In [61]:
sol_1.head()

,building_id,damage_grade
0,a3380c4f75,Grade 2
1,a338a4e653,Grade 5
2,a338a4e6b7,Grade 5
3,a33a6eaa3a,Grade 3
4,a33b073ff6,Grade 5


In [63]:
sol_1.to_csv("sol_1.csv",index=False)

In [105]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators =100,n_jobs=-1,max_features = 0.25)
s = time.time()
print("Training...")
rf.fit(X, y.values.ravel())
print("Training Time" + str(time.time() -s ))
s= time.time()
print("Predicting...")
print("DTree Score- "+ str(rf.score(X_test, y_test)))
print("Predicting Time" + str(time.time() -s ))

Training...
Training Time225.11172151565552
Predicting...
DTree Score- 0.979955574081
Predicting Time53.5569953918457


In [68]:
accuracy = f1_score(y_test, rf.predict(X_test), average='weighted')
print (accuracy)

0.988023631709


In [63]:
#creating the solution dataframe
test_df = test_df.fillna(0)
pred = rf.predict(test_df)
sol_2 = pd.DataFrame({"building_id":test.building_id, "damage_grade":pred})
sol_2['damage_grade'] = 'Grade ' + sol_2['damage_grade'].astype(str)

In [64]:
sol_2.head()

,building_id,damage_grade
0,a3380c4f75,Grade 3
1,a338a4e653,Grade 5
2,a338a4e6b7,Grade 5
3,a33a6eaa3a,Grade 3
4,a33b073ff6,Grade 5


In [65]:
sol_2.to_csv("sol_5.csv",index=False)

In [73]:
print(rf)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)


In [74]:
print(rf.classes_ )

[1 2 3 4 5]


In [77]:
print(rf.feature_importances_ )

[  8.08114429e-02   2.26570363e-02   3.71621870e-03   1.82153052e-03
   7.22684705e-04   1.88749987e-03   2.59036905e-03   2.73163525e-04
   1.21536641e-04   1.71450477e-03   1.47590828e-02   5.68344961e-02
   8.10987906e-02   3.21724147e-03   1.04636909e-02   4.25479099e-03
   2.96773377e-03   2.04815772e-03   6.63121897e-04   1.21852429e-04
   5.51982642e-05   1.47383300e-04   2.46236098e-05   1.95307333e-05
   1.12256697e-05   6.31354865e-04   7.20072839e-03   1.56969050e-01
   7.16417100e-02   7.58636620e-02   3.18369203e-02   1.26467118e-01
   9.96028471e-03   1.17648671e-02   1.25809257e-02   1.41802480e-02
   1.40486859e-02   1.00275993e-02   4.45422488e-03   4.20940508e-03
   1.05324952e-02   2.24879436e-03   1.88860596e-03   4.64993646e-03
   2.95019726e-03   7.40862461e-03   3.94697152e-03   2.24556803e-03
   1.16636127e-03   1.58886086e-03   1.16533893e-01]


In [87]:
X.age_building.value_counts()

15     40598
20     38955
25     33641
10     30147
12     27996
30     27646
5      20994
40     18818
22     17885
8      17442
7      17196
3      17079
35     16795
4      16674
18     16146
6      14614
2      14389
50     14313
14     13550
16     12997
13     12547
1      12322
9      11364
17     11039
11      8914
27      8176
32      8038
28      7904
45      7514
60      7370
       ...  
151        2
121        2
192        1
193        1
195        1
136        1
188        1
185        1
181        1
142        1
178        1
177        1
176        1
132        1
173        1
153        1
148        1
124        1
168        1
167        1
166        1
165        1
164        1
129        1
158        1
156        1
131        1
154        1
133        1
149        1
Name: age_building, dtype: int64